<div style="border: 5px solid purple; padding: 15px; margin: 5px">
<b> Reviewer's comment</b>
    
Hi Brooke, my name is Svetlana (https://hub.tripleten.com/u/6dee602c).  Congratulations on submitting the Final project! 🎉 

    
<div style="border: 5px solid green; padding: 15px; margin: 5px">

- You did a great job on building a good model for churn prediction.


- The notebook demonstrates strong fundamentals: clean data merging, thoughtful preprocessing, train-test splitting, and meaningful metric evaluation.


- The data was merged correctly!


- Great job with feature engineering! 


- `LabelEncoder` is a great choice, well done!

  
    
- Hyperparameters tuning was implemented correctly, good! 
    


</div>
    
<div style="border: 5px solid gold; padding: 15px; margin: 5px">
<b> Reviewer's comment </b>

What can be improved:



- Consider introducing EDA. Distributions and feature correlations may provide helpful context before modeling. 
 



- You can also add the ROC curve for better representation. 



</div>


<hr>
    
<font color='dodgerblue'>**To sum up:**</font> you demonstrated strong analytical and coding skills by preparing the data and training the models. I do not have any questions, so the project can be accepted. Thank you for your diligence on this and other sprints! I am very glad to see your progress 😊 Good luck! 😉
    


</div>

# Interconnect Forcast Churn Project

## Project Description

The telecom operator Interconnect would like to be able to forecast their churn of clients. If it's discovered that a user is planning to leave, they will be offered promotional codes and special plan options. Interconnect's marketing team has collected some of their clientele's personal data, including information about their plans and contracts.

## Work Plan 

Step 1: Analyze service-specific churn patterns during Q4 2019/Q1 2020. This will identify whether phone or internet customers are driving the seasonal churn and help target retention efforts.

Step 2: Analyze month-to-month customers during the seasonal churn period. This will identify if payment flexibility correlates with higher churn rates and reveal demographic or service patterns among this customer segment.

Step 3: Analyze seasonal patterns, customer tenure, and contract types to identify the optimal timing for churn prevention interventions and understand how contract flexibility affects customer retention over time.

Step 4: Synthesize all step findings to bring together all insights for a action plan recommendation.

## Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import lightgbm as lgb

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_curve

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

### Interconnect Services 

Interconnect mainly provides two types of services:

1. Landline communication. The telephone can be connected to several lines simultaneously.
2. Internet. The network can be set up via a telephone line (DSL, _digital subscriber line_) or through a fiber optic cable.

Some other services the company provides include:

- Internet security: antivirus software (_DeviceProtection_) and a malicious website blocker (_OnlineSecurity_)
- A dedicated technical support line (_TechSupport_)
- Cloud file storage and data backup (_OnlineBackup_)
- TV streaming (_StreamingTV_) and a movie directory (_StreamingMovies_)

The clients can choose either a monthly payment or sign a 1- or 2-year contract. They can use various payment methods and receive an electronic invoice after a transaction.

## Load Data

## Internet

In [2]:
interconnect_internet = pd.read_csv('/datasets/final_provider/internet.csv')
interconnect_internet.info()
interconnect_internet.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
3625,5219-YIPTK,Fiber optic,No,Yes,Yes,No,Yes,Yes
2335,2960-NKRSO,Fiber optic,No,Yes,Yes,Yes,No,No
2574,2229-VWQJH,DSL,No,No,Yes,No,No,No


In [3]:
# Checking for missing values 
interconnect_internet.isnull().sum()

customerID          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64

## Phone

In [4]:
interconnect_phone = pd.read_csv('/datasets/final_provider/phone.csv')
interconnect_phone.info()
interconnect_phone.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


In [5]:
# Checking for missing values 
interconnect_phone.isnull().sum()

customerID       0
MultipleLines    0
dtype: int64

## Step 1: Service Churn Patterns

Analyze service-specific churn patterns during Q4 2019/Q1 2020. This will identify whether phone or internet customers are driving the seasonal churn and help target retention efforts.

### Load Contract Data for Churn

In [6]:
interconnect_contract = pd.read_csv('/datasets/final_provider/contract.csv')

In [7]:
# Basic data info
print("Dataset Info:")
print(interconnect_contract.info())
print("\nMissing values:")
print(interconnect_contract.isnull().sum())
print("\nTarget variable distribution:")
print(interconnect_contract['EndDate'].value_counts())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None

Missing values:
customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64

Target variable distribution:
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
20

### Load and Merge All Data

In [8]:
# Load all datasets with correct paths
contract_df = pd.read_csv('/datasets/final_provider/contract.csv')
internet_df = pd.read_csv('/datasets/final_provider/internet.csv')  
phone_df = pd.read_csv('/datasets/final_provider/phone.csv')        
personal_df = pd.read_csv('/datasets/final_provider/personal.csv')  

# merge everything on customerID
df = contract_df.merge(personal_df, on='customerID', how='left')
df = df.merge(internet_df, on='customerID', how='left')
df = df.merge(phone_df, on='customerID', how='left')

print(f"Final merged dataset shape: {df.shape}")

Final merged dataset shape: (7043, 20)


In [9]:
print("EndDate column sample:")
print(df['EndDate'].head(10))
print("\nUnique EndDate values:")
print(df['EndDate'].value_counts())

EndDate column sample:
0                     No
1                     No
2    2019-12-01 00:00:00
3                     No
4    2019-11-01 00:00:00
5    2019-11-01 00:00:00
6                     No
7                     No
8    2019-11-01 00:00:00
9                     No
Name: EndDate, dtype: object

Unique EndDate values:
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: EndDate, dtype: int64


### Creating Target Varible

In [10]:
# Create the churn target variable
df['churn'] = (df['EndDate'] != 'No').astype(int)

# Verify the target distribution
print("Churn distribution:")
print(df['churn'].value_counts())
print(f"\nChurn rate: {df['churn'].mean():.2%}")

Churn distribution:
0    5174
1    1869
Name: churn, dtype: int64

Churn rate: 26.54%


In [11]:
# Check all columns
print("All columns:")
print(df.columns.tolist())

# Check data types
print("\nData types:")
print(df.dtypes)

All columns:
['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines', 'churn']

Data types:
customerID           object
BeginDate            object
EndDate              object
Type                 object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
MultipleLines        object
churn                 int64
dtype: object


In [12]:
# Fixing TotalCharges should be numeric, not object
print("TotalCharges unique values (first 10):")
print(df['TotalCharges'].unique()[:10])

# Checking if there are any non-numeric values
print(f"\nTotalCharges data type: {df['TotalCharges'].dtype}")

TotalCharges unique values (first 10):
['29.85' '1889.5' '108.15' '1840.75' '151.65' '820.5' '1949.4' '301.9'
 '3046.05' '3487.95']

TotalCharges data type: object


In [13]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

Missing values:
customerID             0
BeginDate              0
EndDate                0
Type                   0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
InternetService     1526
OnlineSecurity      1526
OnlineBackup        1526
DeviceProtection    1526
TechSupport         1526
StreamingTV         1526
StreamingMovies     1526
MultipleLines        682
churn                  0
dtype: int64


In [14]:
# Look at the InternetService column (which has 1526 missing values)
print("InternetService column analysis:")
print(df['InternetService'].value_counts(dropna=False))

InternetService column analysis:
Fiber optic    3096
DSL            2421
NaN            1526
Name: InternetService, dtype: int64


In [15]:
# Look at the MultipleLines column (which has 682 missing values)  
print("MultipleLines column analysis:")
print(df['MultipleLines'].value_counts(dropna=False))

MultipleLines column analysis:
No     3390
Yes    2971
NaN     682
Name: MultipleLines, dtype: int64


In [16]:
#Check if customers with no internet service also have NaN for internet-related features
print("Customers with NaN InternetService:")
no_internet = df[df['InternetService'].isna()]
print(f"Count: {len(no_internet)}")

print("\nTheir OnlineSecurity values:")
print(no_internet['OnlineSecurity'].value_counts(dropna=False))

Customers with NaN InternetService:
Count: 1526

Their OnlineSecurity values:
NaN    1526
Name: OnlineSecurity, dtype: int64


In [17]:
# Check customers with NaN MultipleLines
print("Customers with NaN MultipleLines:")
no_phone = df[df['MultipleLines'].isna()]
print(f"Count: {len(no_phone)}")

# Do they have any other phone-related services?
print("\nLet's see what other services they have...")
print("Their InternetService distribution:")
print(no_phone['InternetService'].value_counts(dropna=False))

Customers with NaN MultipleLines:
Count: 682

Let's see what other services they have...
Their InternetService distribution:
DSL    682
Name: InternetService, dtype: int64


In [18]:
# Check if ALL internet-related services are NaN for these customers
print("For customers with no InternetService, check other internet services:")
print("\nOnlineBackup:")
print(no_internet['OnlineBackup'].value_counts(dropna=False))

print("\nDeviceProtection:")
print(no_internet['DeviceProtection'].value_counts(dropna=False))

print("\nTechSupport:")
print(no_internet['TechSupport'].value_counts(dropna=False))

For customers with no InternetService, check other internet services:

OnlineBackup:
NaN    1526
Name: OnlineBackup, dtype: int64

DeviceProtection:
NaN    1526
Name: DeviceProtection, dtype: int64

TechSupport:
NaN    1526
Name: TechSupport, dtype: int64


In [19]:
# Check data types
print(df.dtypes)

# Look at unique values in a specific column
print(df['MonthlyCharges'].value_counts(dropna=False))

customerID           object
BeginDate            object
EndDate              object
Type                 object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
MultipleLines        object
churn                 int64
dtype: object
20.05     61
19.85     45
19.90     44
19.95     44
19.65     43
          ..
87.65      1
35.30      1
114.85     1
56.50      1
97.25      1
Name: MonthlyCharges, Length: 1585, dtype: int64


In [20]:
# Create binary churn target
interconnect_contract['Churn'] = (interconnect_contract['EndDate'] != 'No').astype(int)
print("Churn distribution:")
print(interconnect_contract['Churn'].value_counts())
print(f"Churn rate: {interconnect_contract['Churn'].mean():.2%}")

# Fix TotalCharges data type
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print(f"TotalCharges missing after conversion: {df['TotalCharges'].isnull().sum()}")

# Convert date columns
df['BeginDate'] = pd.to_datetime(df['BeginDate'])
df['EndDate_parsed'] = pd.to_datetime(df['EndDate'], errors='coerce')

Churn distribution:
0    5174
1    1869
Name: Churn, dtype: int64
Churn rate: 26.54%
TotalCharges missing after conversion: 11


In [21]:
# Check if there's a PhoneService column
print("Looking for phone-related columns:")
phone_cols = [col for col in df.columns if 'phone' in col.lower()]
print("Phone columns found:", phone_cols)

# If no phone columns, let's see all columns
print("\nAll columns:")
print(df.columns.tolist())

Looking for phone-related columns:
Phone columns found: []

All columns:
['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines', 'churn', 'EndDate_parsed']


In [22]:
# understanding the MultipleLines pattern
print("MultipleLines value counts:")
print(df['MultipleLines'].value_counts(dropna=False))

print("\nFor customers with NaN MultipleLines, what services do they have?")
no_multiple_lines = df[df['MultipleLines'].isna()]
print(f"Count: {len(no_multiple_lines)}")
print("\nTheir InternetService:")
print(no_multiple_lines['InternetService'].value_counts(dropna=False))

MultipleLines value counts:
No     3390
Yes    2971
NaN     682
Name: MultipleLines, dtype: int64

For customers with NaN MultipleLines, what services do they have?
Count: 682

Their InternetService:
DSL    682
Name: InternetService, dtype: int64


In [23]:
# Replace NaN values with "None" for service-related columns
print("Before cleaning:")
print("InternetService NaN count:", df['InternetService'].isna().sum())
print("MultipleLines NaN count:", df['MultipleLines'].isna().sum())

# Replace NaN with "None" for internet-related services
internet_services = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 
                    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for col in internet_services:
    df[col] = df[col].fillna('None')

# Replace NaN with "None" for MultipleLines
df['MultipleLines'] = df['MultipleLines'].fillna('None')

print("\nAfter cleaning:")
print("InternetService NaN count:", df['InternetService'].isna().sum())
print("MultipleLines NaN count:", df['MultipleLines'].isna().sum())

Before cleaning:
InternetService NaN count: 1526
MultipleLines NaN count: 682

After cleaning:
InternetService NaN count: 0
MultipleLines NaN count: 0


In [24]:
#Verify the cleaning worked 

# Check the new value distributions
print("InternetService after cleaning:")
print(df['InternetService'].value_counts())

print("\nMultipleLines after cleaning:")
print(df['MultipleLines'].value_counts())

InternetService after cleaning:
Fiber optic    3096
DSL            2421
None           1526
Name: InternetService, dtype: int64

MultipleLines after cleaning:
No      3390
Yes     2971
None     682
Name: MultipleLines, dtype: int64


In [25]:
#Checking for any remaining missing values in the dataset
print("Remaining missing values by column:")
missing_summary = df.isnull().sum()
print(missing_summary[missing_summary > 0])

print(f"\nTotal missing values: {df.isnull().sum().sum()}")
print(f"Dataset shape: {df.shape}")

Remaining missing values by column:
TotalCharges        11
EndDate_parsed    5174
dtype: int64

Total missing values: 5185
Dataset shape: (7043, 22)


In [26]:
#checking the relationship between EndDate and EndDate_parsed
print("EndDate value counts:")
print(df['EndDate'].value_counts(dropna=False))

print("\nFor customers with missing EndDate_parsed, what's their EndDate?")
missing_parsed = df[df['EndDate_parsed'].isna()]
print(f"Count with missing EndDate_parsed: {len(missing_parsed)}")
print("Their EndDate values:")
print(missing_parsed['EndDate'].value_counts(dropna=False))

EndDate value counts:
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: EndDate, dtype: int64

For customers with missing EndDate_parsed, what's their EndDate?
Count with missing EndDate_parsed: 5174
Their EndDate values:
No    5174
Name: EndDate, dtype: int64


In [27]:
#investigating those 11 missing TotalCharges
print("Customers with missing TotalCharges:")
missing_charges = df[df['TotalCharges'].isna()]
print(f"Count: {len(missing_charges)}")
print("\nTheir characteristics:")
print("MonthlyCharges:", missing_charges['MonthlyCharges'].describe())
print("EndDate:", missing_charges['EndDate'].value_counts())

Customers with missing TotalCharges:
Count: 11

Their characteristics:
MonthlyCharges: count    11.000000
mean     41.418182
std      23.831484
min      19.700000
25%      20.125000
50%      25.750000
75%      58.975000
max      80.850000
Name: MonthlyCharges, dtype: float64
EndDate: No    11
Name: EndDate, dtype: int64


Active customers are the 5174. The 11 customers who dont have a End Date are new customers. 

In [28]:
# Let's see what makes sense for TotalCharges
print("For the missing TotalCharges customers:")
print("Should we use 0, or their MonthlyCharges, or something else?")
print("\nTheir MonthlyCharges:")
print(missing_charges[['MonthlyCharges']].head())

For the missing TotalCharges customers:
Should we use 0, or their MonthlyCharges, or something else?

Their MonthlyCharges:
      MonthlyCharges
488            52.55
753            20.25
936            80.85
1082           25.75
1340           56.05


In [29]:
# seeing the impact of each approach
print("Dataset size:", len(df))
print("Missing TotalCharges:", df['TotalCharges'].isna().sum())
print("Percentage of data:", (11/7043)*100, "%")

Dataset size: 7043
Missing TotalCharges: 11
Percentage of data: 0.1561834445548772 %


- First month: TotalCharges = 0 (they haven't been billed yet)
- Next month: Either they churn (leave) OR they get charged their MonthlyCharges

In [30]:
# Fill missing TotalCharges with 0 (representing new customers)
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# Verify the change
print("Missing TotalCharges after filling:", df['TotalCharges'].isna().sum())
print("TotalCharges = 0 count:", (df['TotalCharges'] == 0).sum())

Missing TotalCharges after filling: 0
TotalCharges = 0 count: 11


- EndDate == 'No' means the customer is still active (not churned)
- EndDate != 'No' means the customer has churned

In [31]:
# Create binary target variable
# 1 = churned, 0 = still active
df['Churn'] = (df['EndDate'] != 'No').astype(int)

# check the distribution
print("Churn distribution:")
print(df['Churn'].value_counts())
print("\nChurn rate:")
print(df['Churn'].mean())

Churn distribution:
0    5174
1    1869
Name: Churn, dtype: int64

Churn rate:
0.2653698707936959


- 5,174 customers stayed (Churn = 0)
- 1,869 customers left (Churn = 1)
- Overall churn rate: 26.5%

## Step 2: Feature Selection & Data Exploration

Analyze month-to-month customers during the seasonal churn period. This will identify if payment flexibility correlates with higher churn rates and reveal demographic or service patterns among this customer segment.

In [32]:
# exploring what columns we have
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nData types:")
print(df.dtypes)

Dataset shape: (7043, 23)

Column names:
['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines', 'churn', 'EndDate_parsed', 'Churn']

Data types:
customerID                  object
BeginDate           datetime64[ns]
EndDate                     object
Type                        object
PaperlessBilling            object
PaymentMethod               object
MonthlyCharges             float64
TotalCharges               float64
gender                      object
SeniorCitizen                int64
Partner                     object
Dependents                  object
InternetService             object
OnlineSecurity              object
OnlineBackup                object
DeviceProtection            object
TechSupport                 object
Stream

#### New Customer Feature

In [33]:
# Calculate tenure in months
df['tenure'] = ((pd.to_datetime('2020-02-01') - df['BeginDate']).dt.days / 30.44).round().astype(int)

In [34]:
# Define new customers (< 6 months tenure)
df['New_customer'] = (df['tenure'] < 6).astype(int)

# Check the distribution
print("New customer distribution:")
print(df['New_customer'].value_counts())

New customer distribution:
0    5868
1    1175
Name: New_customer, dtype: int64


### Premium Services Feature

In [35]:
df['Premium_services'] = ((df['InternetService'] != 'No') & 
                         (df['MultipleLines'] == 'Yes')).astype(int)

print("Premium services distribution:")
print(df['Premium_services'].value_counts())

Premium services distribution:
0    4072
1    2971
Name: Premium_services, dtype: int64


In [36]:
# Check the date columns
print("BeginDate sample:")
print(df['BeginDate'].head())
print("\nEndDate sample:")
print(df['EndDate'].head())

BeginDate sample:
0   2020-01-01
1   2017-04-01
2   2019-10-01
3   2016-05-01
4   2019-09-01
Name: BeginDate, dtype: datetime64[ns]

EndDate sample:
0                     No
1                     No
2    2019-12-01 00:00:00
3                     No
4    2019-11-01 00:00:00
Name: EndDate, dtype: object


In [37]:
# Check the EndDate column more closely
print("EndDate value counts:")
print(df['EndDate'].value_counts())
print("\nEndDate data type:", df['EndDate'].dtype)

EndDate value counts:
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: EndDate, dtype: int64

EndDate data type: object


In [38]:
# Load the contract data
contract_df = pd.read_csv('/datasets/final_provider/contract.csv')

# Basic exploration - use contract_df instead of contract
print("Contract data shape:", contract_df.shape)
print("\nFirst few rows:")
print(contract_df.head())
print("\nColumn names:")
print(contract_df.columns.tolist())
print("\nData types:")
print(contract_df.dtypes)

Contract data shape: (7043, 8)

First few rows:
   customerID   BeginDate              EndDate            Type  \
0  7590-VHVEG  2020-01-01                   No  Month-to-month   
1  5575-GNVDE  2017-04-01                   No        One year   
2  3668-QPYBK  2019-10-01  2019-12-01 00:00:00  Month-to-month   
3  7795-CFOCW  2016-05-01                   No        One year   
4  9237-HQITU  2019-09-01  2019-11-01 00:00:00  Month-to-month   

  PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  
0              Yes           Electronic check           29.85        29.85  
1               No               Mailed check           56.95       1889.5  
2              Yes               Mailed check           53.85       108.15  
3               No  Bank transfer (automatic)           42.30      1840.75  
4              Yes           Electronic check           70.70       151.65  

Column names:
['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling', 'PaymentMet

In [39]:
# Looking at the date columns structure
print("BeginDate examples:")
print(contract_df['BeginDate'].head(10))
print("\nEndDate examples:")
print(contract_df['EndDate'].head(10))
print("\nEndDate unique values (first 20):")
print(contract_df['EndDate'].unique()[:20])

BeginDate examples:
0    2020-01-01
1    2017-04-01
2    2019-10-01
3    2016-05-01
4    2019-09-01
5    2019-03-01
6    2018-04-01
7    2019-04-01
8    2017-07-01
9    2014-12-01
Name: BeginDate, dtype: object

EndDate examples:
0                     No
1                     No
2    2019-12-01 00:00:00
3                     No
4    2019-11-01 00:00:00
5    2019-11-01 00:00:00
6                     No
7                     No
8    2019-11-01 00:00:00
9                     No
Name: EndDate, dtype: object

EndDate unique values (first 20):
['No' '2019-12-01 00:00:00' '2019-11-01 00:00:00' '2019-10-01 00:00:00'
 '2020-01-01 00:00:00']


In [40]:
# First, convert BeginDate to datetime
contract_df['BeginDate'] = pd.to_datetime(contract_df['BeginDate'])

# Check the conversion
print("BeginDate after conversion:")
print(contract_df['BeginDate'].head())
print(f"Data type: {contract_df['BeginDate'].dtype}")

BeginDate after conversion:
0   2020-01-01
1   2017-04-01
2   2019-10-01
3   2016-05-01
4   2019-09-01
Name: BeginDate, dtype: datetime64[ns]
Data type: datetime64[ns]


In [41]:
# Convert BeginDate to datetime 
contract_df['BeginDate'] = pd.to_datetime(contract_df['BeginDate'])

# Create a proper EndDate column for calculation
# Replace "No" with 2020-01-01 for active customers
contract_df['EndDate_calc'] = contract_df['EndDate'].replace('No', '2020-01-01')

# Convert the calculation column to datetime
contract_df['EndDate_calc'] = pd.to_datetime(contract_df['EndDate_calc'])

# Checking work
print("Sample of the calculation columns:")
print(contract_df[['customerID', 'BeginDate', 'EndDate', 'EndDate_calc']].head(10))

Sample of the calculation columns:
   customerID  BeginDate              EndDate EndDate_calc
0  7590-VHVEG 2020-01-01                   No   2020-01-01
1  5575-GNVDE 2017-04-01                   No   2020-01-01
2  3668-QPYBK 2019-10-01  2019-12-01 00:00:00   2019-12-01
3  7795-CFOCW 2016-05-01                   No   2020-01-01
4  9237-HQITU 2019-09-01  2019-11-01 00:00:00   2019-11-01
5  9305-CDSKC 2019-03-01  2019-11-01 00:00:00   2019-11-01
6  1452-KIOVK 2018-04-01                   No   2020-01-01
7  6713-OKOMC 2019-04-01                   No   2020-01-01
8  7892-POOKP 2017-07-01  2019-11-01 00:00:00   2019-11-01
9  6388-TABGU 2014-12-01                   No   2020-01-01


In [42]:
# Calculate tenure in months
contract_df['tenure_months'] = ((contract_df['EndDate_calc'] - contract_df['BeginDate']).dt.days / 30.44).round().astype(int)

# Check the results
print("Tenure calculation results:")
print(contract_df[['customerID', 'BeginDate', 'EndDate_calc', 'tenure_months']].head(10))

# Let's also check the tenure distribution
print(f"\nTenure statistics:")
print(contract_df['tenure_months'].describe())

Tenure calculation results:
   customerID  BeginDate EndDate_calc  tenure_months
0  7590-VHVEG 2020-01-01   2020-01-01              0
1  5575-GNVDE 2017-04-01   2020-01-01             33
2  3668-QPYBK 2019-10-01   2019-12-01              2
3  7795-CFOCW 2016-05-01   2020-01-01             44
4  9237-HQITU 2019-09-01   2019-11-01              2
5  9305-CDSKC 2019-03-01   2019-11-01              8
6  1452-KIOVK 2018-04-01   2020-01-01             21
7  6713-OKOMC 2019-04-01   2020-01-01              9
8  7892-POOKP 2017-07-01   2019-11-01             28
9  6388-TABGU 2014-12-01   2020-01-01             61

Tenure statistics:
count    7043.000000
mean       31.636519
std        24.407449
min        -1.000000
25%         8.000000
50%        28.000000
75%        55.000000
max        72.000000
Name: tenure_months, dtype: float64


In [43]:
contract_df['quick_churn'] = contract_df['EndDate'] == contract_df['BeginDate']

In [44]:
negative_tenure = contract_df[contract_df['tenure_months'] == -1]
print("Customers with -1 tenure:")
print(negative_tenure[['customerID', 'BeginDate', 'EndDate_calc', 'tenure_months']])

Customers with -1 tenure:
      customerID  BeginDate EndDate_calc  tenure_months
488   4472-LVYGI 2020-02-01   2020-01-01             -1
753   3115-CZMZD 2020-02-01   2020-01-01             -1
936   5709-LVOEQ 2020-02-01   2020-01-01             -1
1082  4367-NUYAO 2020-02-01   2020-01-01             -1
1340  1371-DWPAZ 2020-02-01   2020-01-01             -1
3331  7644-OMVMY 2020-02-01   2020-01-01             -1
3826  3213-VVOLG 2020-02-01   2020-01-01             -1
4380  2520-SGTTA 2020-02-01   2020-01-01             -1
5218  2923-ARZLG 2020-02-01   2020-01-01             -1
6670  4075-WKNIU 2020-02-01   2020-01-01             -1
6754  2775-SEFEE 2020-02-01   2020-01-01             -1


In [45]:
negative_tenure['days_diff'] = (negative_tenure['EndDate_calc'] - negative_tenure['BeginDate']).dt.days
print(negative_tenure[['customerID', 'BeginDate', 'EndDate_calc', 'days_diff', 'tenure_months']])

      customerID  BeginDate EndDate_calc  days_diff  tenure_months
488   4472-LVYGI 2020-02-01   2020-01-01        -31             -1
753   3115-CZMZD 2020-02-01   2020-01-01        -31             -1
936   5709-LVOEQ 2020-02-01   2020-01-01        -31             -1
1082  4367-NUYAO 2020-02-01   2020-01-01        -31             -1
1340  1371-DWPAZ 2020-02-01   2020-01-01        -31             -1
3331  7644-OMVMY 2020-02-01   2020-01-01        -31             -1
3826  3213-VVOLG 2020-02-01   2020-01-01        -31             -1
4380  2520-SGTTA 2020-02-01   2020-01-01        -31             -1
5218  2923-ARZLG 2020-02-01   2020-01-01        -31             -1
6670  4075-WKNIU 2020-02-01   2020-01-01        -31             -1
6754  2775-SEFEE 2020-02-01   2020-01-01        -31             -1


In [46]:
contract_df['tenure_months'] = np.where(contract_df['tenure_months'] < 0, 0, contract_df['tenure_months'])

In [47]:
# Check if there are any negative values left
print("Negative tenure values remaining:", (contract_df['tenure_months'] < 0).sum())
print("Zero tenure values:", (contract_df['tenure_months'] == 0).sum())

Negative tenure values remaining: 0
Zero tenure values: 244


In [48]:
contract_df['tenure_months'].describe()

count    7043.000000
mean       31.638080
std        24.405393
min         0.000000
25%         8.000000
50%        28.000000
75%        55.000000
max        72.000000
Name: tenure_months, dtype: float64

Key Insights from tenure distribution:

- Total customers: 7,043
- Average tenure: ~32 months (about 2.7 years)
- Range: 0 to 72 months (0 to 6 years)
- Median: 28 months; meaning half thhe customers have been with the company less than 2.3 years

The distribution shows:

- 25% of customers have 8 months or less (newer customers)
- 50% of customers have 28 months or less
- 75% of customers have 55 months or less
- 244 customers (3.5%) are brand new with 0 months

### Step 3: Seasonal Patterns, Customer Churn

Analyze seasonal patterns, customer tenure, and contract types to identify the optimal timing for churn prevention interventions and understand how contract flexibility affects customer retention over time.

In [49]:
# Create a binary churn indicator (1 = churned, 0 = active)
contract_df['churn_binary'] = (contract_df['EndDate'] != 'No').astype(int)

In [50]:
# Create tenure groups for better analysis
def categorize_tenure(months):
    if months == 0:
        return '0 months (New)'
    elif months <= 12:
        return '1-12 months (Early)'
    elif months <= 24:
        return '13-24 months (Developing)'
    elif months <= 36:
        return '25-36 months (Established)'
    elif months <= 48:
        return '37-48 months (Loyal)'
    else:
        return '49+ months (Very Loyal)'

# Apply the function to create tenure_group column
contract_df['tenure_group'] = contract_df['tenure_months'].apply(categorize_tenure)

In [51]:
# Calculate churn rate by tenure group
tenure_churn = contract_df.groupby('tenure_group').agg({
    'churn_binary': ['count', 'sum', 'mean']  # Use the binary column instead
}).round(3)

print("Churn Analysis by Tenure Group:")
print(tenure_churn)

Churn Analysis by Tenure Group:
                           churn_binary             
                                  count   sum   mean
tenure_group                                        
0 months (New)                      244     0  0.000
1-12 months (Early)                2013  1037  0.515
13-24 months (Developing)          1009   294  0.291
25-36 months (Established)          826   180  0.218
37-48 months (Loyal)                763   145  0.190
49+ months (Very Loyal)            2188   213  0.097


Tenure Pattern Analysis:
- Highest Risk: Early customers (1-12 months) with 51.5% churn rate
- Stabilization Point: Around 25+ months, churn drops significantly
- Most Stable: Very loyal customers (49+ months) with only 9.7% churn

**Early Intervention Window: The first 12 months are critical! This is when you lose over half your customers.** 

In [52]:
# Extract month from BeginDate for seasonal analysis
contract_df['begin_month'] = pd.to_datetime(contract_df['BeginDate']).dt.month

# Calculate churn rate by month
monthly_churn = contract_df.groupby('begin_month').agg({
    'churn_binary': ['count', 'sum', 'mean']
}).round(3)

print("Churn Analysis by Start Month:")
print(monthly_churn)

Churn Analysis by Start Month:
            churn_binary            
                   count  sum   mean
begin_month                         
1                    623   94  0.151
2                    804  122  0.152
3                    590  107  0.181
4                    511   98  0.192
5                    494  126  0.255
6                    522  132  0.253
7                    518  142  0.274
8                    532  151  0.284
9                    623  257  0.413
10                   601  212  0.353
11                   621  224  0.361
12                   604  204  0.338


In [53]:
# Create seasonal categories
def categorize_season(month):
    if month in [10, 11, 12, 1]:  # identified seasonal months
        return 'High Season'
    else:
        return 'Regular Season'

contract_df['season_category'] = contract_df['begin_month'].apply(categorize_season)

# Analyze churn by season
seasonal_churn = contract_df.groupby('season_category').agg({
    'churn_binary': ['count', 'sum', 'mean']
}).round(3)

print("Churn Analysis by Season:")
print(seasonal_churn)

Churn Analysis by Season:
                churn_binary             
                       count   sum   mean
season_category                          
High Season             2449   734  0.300
Regular Season          4594  1135  0.247


In [54]:
# Combined analysis: Season + Tenure
seasonal_tenure_churn = contract_df.groupby(['season_category', 'tenure_group']).agg({
    'churn_binary': ['count', 'sum', 'mean']
}).round(3)

print("Combined Seasonal + Tenure Analysis:")
print(seasonal_tenure_churn)

Combined Seasonal + Tenure Analysis:
                                           churn_binary            
                                                  count  sum   mean
season_category tenure_group                                       
High Season     0 months (New)                      233    0  0.000
                1-12 months (Early)                 864  479  0.554
                13-24 months (Developing)           319   97  0.304
                25-36 months (Established)          274   56  0.204
                37-48 months (Loyal)                230   40  0.174
                49+ months (Very Loyal)             529   62  0.117
Regular Season  0 months (New)                       11    0  0.000
                1-12 months (Early)                1149  558  0.486
                13-24 months (Developing)           690  197  0.286
                25-36 months (Established)          552  124  0.225
                37-48 months (Loyal)                533  105  0.197
           

**Seasonal Effect Confirmation:**
- High Season (Oct-Nov-Dec-Jan): Early customers have 55.4% churn rate
- Regular Season: Early customers have 48.6% churn rate
- Early tenure (1-12 months) + High season = 55.4% churn rate

**Seasonal Impact Across All Tenure Groups:**

Every tenure group shows higher churn during high season:
- Developing: 30.4% vs 28.6%
- Established: 20.4% vs 22.5% (interesting reversal!)
- Loyal: 17.4% vs 19.7%
- Very Loyal: 11.7% vs 9.1%

**Developing customers (13-24 months): Mean ~$300, higher churn risk.**

**Very loyal customers (49+ months): Mean ~$100, low churn risk.**

### Ladder pricing strategy 

**Tiered Discount Structure Could Look Like:**
    
- Year 1-2 (Developing): Full price (~$300)

- Year 2-3: 10% loyalty discount (~$270)

- Year 3-4: 20% loyalty discount (~$240)

- Year 4+: Maximum loyalty rate (~$100-150)

In [55]:
# Step-by-step merging
merged_df = contract_df.merge(internet_df, on='customerID', how='left')
final_df = merged_df.merge(phone_df, on='customerID', how='left')

In [56]:
# Check the basic info for each
print("Contract dataset shape:", contract_df.shape)
print("Internet dataset shape:", internet_df.shape)
print("Phone dataset shape:", phone_df.shape)

Contract dataset shape: (7043, 15)
Internet dataset shape: (5517, 8)
Phone dataset shape: (6361, 2)


In [57]:
print("Phone customerID info:")
print(f"- Unique customers: {phone_df['customerID'].nunique()}")
print(f"- Any missing customerIDs: {phone_df['customerID'].isnull().sum()}")

Phone customerID info:
- Unique customers: 6361
- Any missing customerIDs: 0


In [58]:
# Check if customerID exists and look for issues
print("Contract customerID info:")
print(f"- Unique customers: {contract_df['customerID'].nunique()}")
print(f"- Any missing customerIDs: {contract_df['customerID'].isnull().sum()}")

print("\nInternet customerID info:")
print(f"- Unique customers: {internet_df['customerID'].nunique()}")
print(f"- Any missing customerIDs: {internet_df['customerID'].isnull().sum()}")

Contract customerID info:
- Unique customers: 7043
- Any missing customerIDs: 0

Internet customerID info:
- Unique customers: 5517
- Any missing customerIDs: 0


In [59]:

# Extract month and season from BeginDate
contract_df['month'] = contract_df['BeginDate'].dt.month
contract_df['season'] = contract_df['BeginDate'].dt.month.map({12:4, 1:4, 2:4, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3, 10:3, 11:3})


In [60]:
print("BeginDate range:")
print(f"Earliest: {contract_df['BeginDate'].min()}")
print(f"Latest: {contract_df['BeginDate'].max()}")

print("\nEndDate range (excluding 'No'):")
end_dates = contract_df[contract_df['EndDate'] != 'No']['EndDate']
print(f"Earliest: {end_dates.min()}")
print(f"Latest: {end_dates.max()}")

BeginDate range:
Earliest: 2013-10-01 00:00:00
Latest: 2020-02-01 00:00:00

EndDate range (excluding 'No'):
Earliest: 2019-10-01 00:00:00
Latest: 2020-01-01 00:00:00


In [61]:

# Use BeginDate for chronological split
# Split at 2018-01-01 (adjust based on your preference)
train_data = contract_df[contract_df['BeginDate'] < '2018-01-01']
test_data = contract_df[contract_df['BeginDate'] >= '2018-01-01']

print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")


Training set size: 3812
Test set size: 3231


In [62]:
target = (contract_df['EndDate'] != 'No').astype(int)  # 1 = churned, 0 = active

In [63]:
print("Target distribution:")
print(target.value_counts())
print(f"Churn rate: {target.mean():.2%}")

Target distribution:
0    5174
1    1869
Name: EndDate, dtype: int64
Churn rate: 26.54%


- Training set: 3,812 samples
- Test set: 3,231 samples
- Overall churn rate: 26.54% (1,869 churned out of 7,043 total customers)

In [64]:
contract_df.columns

Index(['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'EndDate_calc',
       'tenure_months', 'quick_churn', 'churn_binary', 'tenure_group',
       'begin_month', 'season_category', 'month', 'season'],
      dtype='object')

In [65]:
contract_df.customerID

0       7590-VHVEG
1       5575-GNVDE
2       3668-QPYBK
3       7795-CFOCW
4       9237-HQITU
           ...    
7038    6840-RESVB
7039    2234-XADUH
7040    4801-JZAZL
7041    8361-LTMKD
7042    3186-AJIEK
Name: customerID, Length: 7043, dtype: object

In [66]:
internet_df.columns

Index(['customerID', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'],
      dtype='object')

In [67]:
internet_df.customerID

0       7590-VHVEG
1       5575-GNVDE
2       3668-QPYBK
3       7795-CFOCW
4       9237-HQITU
           ...    
5512    6840-RESVB
5513    2234-XADUH
5514    4801-JZAZL
5515    8361-LTMKD
5516    3186-AJIEK
Name: customerID, Length: 5517, dtype: object

In [68]:
phone_df.columns

Index(['customerID', 'MultipleLines'], dtype='object')

In [69]:
phone_df.customerID

0       5575-GNVDE
1       3668-QPYBK
2       9237-HQITU
3       9305-CDSKC
4       1452-KIOVK
           ...    
6356    2569-WGERO
6357    6840-RESVB
6358    2234-XADUH
6359    8361-LTMKD
6360    3186-AJIEK
Name: customerID, Length: 6361, dtype: object

In [70]:
personal_df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents'], dtype='object')

In [71]:
personal_df.customerID

0       7590-VHVEG
1       5575-GNVDE
2       3668-QPYBK
3       7795-CFOCW
4       9237-HQITU
           ...    
7038    6840-RESVB
7039    2234-XADUH
7040    4801-JZAZL
7041    8361-LTMKD
7042    3186-AJIEK
Name: customerID, Length: 7043, dtype: object

In [72]:
# Fill missing values to No Service
df['InternetService'] = df['InternetService'].fillna('No Service')
df['OnlineSecurity'] = df['OnlineSecurity'].fillna('No Service')
df['TechSupport'] = df['TechSupport'].fillna('No Service')
df['OnlineBackup'] = df['OnlineBackup'].fillna('No Service')
df['DeviceProtection'] = df['DeviceProtection'].fillna('No Service')
df['StreamingTV'] = df['StreamingTV'].fillna('No Service')
df['StreamingMovies'] = df['StreamingMovies'].fillna('No Service')
df['MultipleLines'] = df['MultipleLines'].fillna('No Service')

In [73]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
7038    False
7039    False
7040    False
7041    False
7042    False
Length: 7043, dtype: bool

In [74]:
df.isnull().sum()

customerID             0
BeginDate              0
EndDate                0
Type                   0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
InternetService        0
OnlineSecurity         0
OnlineBackup           0
DeviceProtection       0
TechSupport            0
StreamingTV            0
StreamingMovies        0
MultipleLines          0
churn                  0
EndDate_parsed      5174
Churn                  0
tenure                 0
New_customer           0
Premium_services       0
dtype: int64

**Model Pipeline Plan:**
- Logical Regression (baseline/sanity check)
- Random Forest (with hyperparameter tuning)
- LightGBM (with hyperparameter tuning)
- CatBoost (final model)

In [75]:
# Target variable
target = df['churn'] 

# Features (exclude non-predictive columns)
features = df.drop(['customerID', 'BeginDate', 'EndDate', 'EndDate_parsed', 'churn', 'Churn'], axis=1)

In [76]:
# Create a copy for preprocessing
features_encoded = features.copy()

# List of categorical columns to encode
categorical_cols = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender', 
                   'Partner', 'Dependents', 'InternetService', 'OnlineSecurity',
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                   'StreamingTV', 'StreamingMovies', 'MultipleLines']

# Encode categorical variables
le = LabelEncoder()
for col in categorical_cols:
    features_encoded[col] = le.fit_transform(features_encoded[col])

# Check the result
print("Features encoded successfully!")
print(features_encoded.dtypes)

Features encoded successfully!
Type                  int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
MultipleLines         int64
tenure                int64
New_customer          int64
Premium_services      int64
dtype: object


In [77]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    features_encoded, target, test_size=0.2, random_state=42, stratify=target
)

# Scale the features 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [78]:
# Train the baseline model
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

# Calculate AUC-ROC
from sklearn.metrics import roc_auc_score
auc_lr = roc_auc_score(y_test, y_pred_proba_lr)

print(f"Logistic Regression AUC-ROC: {auc_lr:.4f}")

Logistic Regression AUC-ROC: 0.8316


In [79]:
features.dtypes

Type                 object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
MultipleLines        object
tenure                int64
New_customer          int64
Premium_services      int64
dtype: object

In [80]:
# default parameters
rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict_proba(X_test)[:, 1]
auc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"Random Forest AUC-ROC: {auc_rf:.4f}")

Random Forest AUC-ROC: 0.8746


In [81]:
# Define parameter distributions
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

In [82]:
rf_random = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter combinations to try
    cv=5,       # 5-fold cross-validation
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1   # Use all available cores
)

In [83]:
# LightGBM parameter distributions
lgb_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10, -1],  # -1 means no limit
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [31, 50, 100, 200],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [84]:
lgb_random = RandomizedSearchCV(
    lgb.LGBMClassifier(random_state=42, verbose=-1),
    param_distributions=lgb_param_dist,
    n_iter=50,
    cv=5,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1
)

In [85]:
lgb_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 10, -1],  # -1 means no limit
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [31, 50, 100, 200],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [86]:
# Fit the RandomizedSearchCV
lgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(random_state=42, verbose=-1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.8, 0.9, 1.0],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 10, -1],
                                        'min_child_samples': [10, 20, 30],
                                        'n_estimators': [100, 200, 300, 500],
                                        'num_leaves': [31, 50, 100, 200],
                                        'subsample': [0.8, 0.9, 1.0]},
                   random_state=42, scoring='roc_auc')

In [87]:
print("Best parameters:", lgb_random.best_params_)
print("Best cross-validation score:", lgb_random.best_score_)

Best parameters: {'subsample': 0.9, 'num_leaves': 100, 'n_estimators': 300, 'min_child_samples': 30, 'max_depth': 5, 'learning_rate': 0.2, 'colsample_bytree': 0.9}
Best cross-validation score: 0.9346840769060604


In [88]:
# Get the best model
best_lgb = lgb_random.best_estimator_

# Make predictions
lgb_pred_proba = best_lgb.predict_proba(X_test)[:, 1]

# Calculate AUC-ROC
from sklearn.metrics import roc_auc_score
lgb_auc = roc_auc_score(y_test, lgb_pred_proba)
print(f"LightGBM AUC-ROC: {lgb_auc:.4f}")

LightGBM AUC-ROC: 0.9388
